In [13]:
! mkdir Data
! wget -O Data/gene_expression.csv.gz "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE96058&format=file&file=GSE96058%5Fgene%5Fexpression%5F3273%5Fsamples%5Fand%5F136%5Freplicates%5Ftransformed%2Ecsv%2Egz"
! wget -O Data/meta_1.txt.gz "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE96nnn/GSE96058/matrix/GSE96058-GPL11154_series_matrix.txt.gz"
! wget -O Data/meta_2.txt.gz "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE96nnn/GSE96058/matrix/GSE96058-GPL18573_series_matrix.txt.gz"
! wget -O ../Data/transcript_expression.csv.gz "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE96058&format=file&file=GSE96058%5Ftranscript%5Fexpression%5F3273%5Fsamples%5Fand%5F136%5Freplicates%2Ecsv%2Egz"
! wget -O ../Data/known_genes.gtf.gz "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE96058&format=file&file=GSE96058%5FUCSC%5Fhg38%5FknownGenes%5F22sep2014%2Egtf%2Egz"
!gunzip Data/*

--2025-07-01 10:51:21--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE96nnn/GSE96058/matrix/GSE96058-GPL11154_series_matrix.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218719 (214K) [application/x-gzip]
Saving to: ‘Data/meta_1.txt.gz’

Data/meta_1.txt.gz  100%[===================>] 213.59K   640KB/s    in 0.3s    

2025-07-01 10:51:22 (640 KB/s) - ‘Data/meta_1.txt.gz’ saved [218719/218719]

--2025-07-01 10:51:22--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE96nnn/GSE96058/matrix/GSE96058-GPL18573_series_matrix.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.31, 130.14.250.7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42331 (41K) [

In [189]:
import pandas as pd
import numpy as np
import seaborn as sns
import anndata as ad
import os

In [190]:
df=pd.read_csv('Data/gene_expression.csv', index_col=0)

In [191]:
df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F2974repl,F3006repl,F3028repl,F3057repl,F3058repl,F3085repl,F3127repl,F3135repl,F3250repl,F3265repl
5_8S_rRNA,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
5S_rRNA,4.911099,-3.321928,-3.321928,3.656393,4.190104,2.556304,2.590351,5.691788,-3.321928,3.223401,...,4.084251,2.287523,3.205371,6.965451,-3.321928,4.531984,-3.321928,5.097337,1.895737,2.954997
6M1-18,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
7M1-2,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
7SK,-0.539253,-0.576620,-1.651323,0.126633,0.783715,-1.759556,-1.033968,-0.129513,0.494308,0.749841,...,0.233159,-0.026301,1.249039,1.651384,0.128604,-0.594062,-0.228405,0.239723,2.074377,0.867608


In [192]:
replicates = [c for c in df.columns if c.endswith('repl')]
for repl in replicates:
    og = repl[:-4]
    df[og] = df[[og,repl]].mean(axis=1)
    del df[repl]

In [198]:
df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F3264,F3265,F3266,F3267,F3268,F3269,F3270,F3271,F3272,F3273
5_8S_rRNA,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
5S_rRNA,4.911099,-3.321928,-3.321928,3.656393,4.190104,2.556304,2.590351,1.184930,-3.321928,3.223401,...,4.120153,-0.183466,-3.321928,2.594778,2.747615,-3.321928,3.715948,4.355923,-3.321928,-3.321928
6M1-18,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
7M1-2,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
7SK,-0.539253,-0.576620,-1.651323,0.126633,0.783715,-1.759556,-1.033968,-0.356898,0.494308,0.749841,...,0.656767,0.546187,-0.318436,-0.315955,-0.979706,0.915773,0.364539,0.258764,1.044011,0.496374


In [199]:
df.shape

(30865, 3273)

In [214]:
def get_meta(path):
    with open(path) as f:
        lines=f.readlines()    
    meta_data = [line for line in lines if line.startswith("!Sample")]
    meta_dict = {}
    for line in meta_data:
        if line.startswith("!Sample_title"):
            samples=line.replace("!Sample_title","").strip().split('\t')
            samples=[c.strip('"') for c in samples]
            meta_dict['Sample'] =samples
        elif line.startswith("!Sample_characteristics_ch1"):
            parts = line.strip().split('\t')
            values = [v.strip('"') for v in parts[1:]]  
            key = values[0].split(':')[0].strip()
            data = [v.split(':')[1].strip() for v in values]
            meta_dict[key]=data
    metadata = pd.DataFrame(meta_dict)
    return metadata

In [215]:
metadata = pd.concat([get_meta(f"Data/{p}") for p in os.listdir("Data/") if p.startswith("meta")],axis=0)

In [216]:
metadata=metadata[metadata['Sample'].isin(df.columns)]
df = df[metadata['Sample']]
metadata=metadata.set_index('Sample')

In [217]:
metadata.columns

Index(['scan-b external id', 'instrument model', 'age at diagnosis',
       'tumor size', 'lymph node group', 'lymph node status', 'er status',
       'pgr status', 'her2 status', 'ki67 status', 'nhg', 'er prediction mgc',
       'pgr prediction mgc', 'her2 prediction mgc', 'ki67 prediction mgc',
       'nhg prediction mgc', 'er prediction sgc', 'pgr prediction sgc',
       'her2 prediction sgc', 'ki67 prediction sgc', 'pam50 subtype',
       'overall survival days', 'overall survival event', 'endocrine treated',
       'chemo treated'],
      dtype='object')

In [218]:
X = df.T
adata = ad.AnnData(X=X, obs=metadata)

In [219]:
adata

AnnData object with n_obs × n_vars = 3273 × 30865
    obs: 'scan-b external id', 'instrument model', 'age at diagnosis', 'tumor size', 'lymph node group', 'lymph node status', 'er status', 'pgr status', 'her2 status', 'ki67 status', 'nhg', 'er prediction mgc', 'pgr prediction mgc', 'her2 prediction mgc', 'ki67 prediction mgc', 'nhg prediction mgc', 'er prediction sgc', 'pgr prediction sgc', 'her2 prediction sgc', 'ki67 prediction sgc', 'pam50 subtype', 'overall survival days', 'overall survival event', 'endocrine treated', 'chemo treated'

In [220]:
adata.write('Data/dataset.h5ad')